# Lesson 1: Advanced RAG Pipeline

In [3]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

#openai.api_key = "sk-proj-QqtjsCZJacoM5OMr1mO3T3BlbkFJTyir0czUVQXsP4FHwMrU"

In [4]:
import pkg_resources

dists = [str(d).replace(" ","==") for d in pkg_resources.working_set]
for i in dists:
    print(i)

certifi==2023.11.17
jsonschema-specifications==2023.11.1
regex==2023.10.3
fsspec==2023.10.0
pytz==2023.3.post1
tzdata==2023.3
pywin32==306
setuptools==65.5.1
pyzmq==25.1.1
pip==24.0
packaging==23.2
argon2-cffi==23.1.0
attrs==23.1.0
argon2-cffi-bindings==21.2.0
isoduration==20.11.0
Faker==20.1.0
pyarrow==14.0.1
rich==13.7.0
nvidia-nvjitlink-cu12==12.3.101
nvidia-cuda-cupti-cu12==12.1.105
nvidia-cuda-nvrtc-cu12==12.1.105
nvidia-cuda-runtime-cu12==12.1.105
nvidia-nvtx-cu12==12.1.105
nvidia-cublas-cu12==12.1.3.1
nvidia-cusparse-cu12==12.1.0.106
nvidia-cusolver-cu12==11.4.5.107
nvidia-cufft-cu12==11.0.2.54
pymdown-extensions==10.5
nvidia-curand-cu12==10.3.2.106
more-itertools==10.1.0
Pillow==10.1.0
ipython==8.18.0
nvidia-cudnn-cu12==8.9.4.25
jupyter-client==8.6.0
tenacity==8.2.3
click==8.1.7
ipywidgets==8.1.0
python-slugify==8.0.1
nbconvert==7.11.0
overrides==7.4.0
ipykernel==6.26.0
importlib-metadata==6.8.0
notebook==6.4.13
tornado==6.3.3
bleach==6.1.0
multidict==6.0.4
PyYAML==6.0.1
psutil

In [5]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./CELEX_32016R0679_EN_TXT.pdf"]
).load_data()

In [6]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

88 

<class 'llama_index.schema.Document'>
Doc ID: 707847a1-5dca-4d2a-8a89-4a751f7c09df
Text: I  (Legislativ e acts)  REGUL ATIONS  REGUL ATION (EU) 2016/679
OF THE EUR OPEAN PARLIAMENT AND OF THE COUNCIL  of 27 Apr il 2016  on
the protection of natural persons with regard to the processing of
personal data and on the free  movement of such data, and repealing
Directiv e 95/46/EC (General Data Protection Regulation)  (Text with
EEA relev...


## Basic RAG pipeline

In [7]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [14]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms.ollama import Ollama
from llama_index.llms import OpenAI

#llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
llm = Ollama(model="Saul-Instruct-v1")
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)
index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

In [15]:
query_engine = index.as_query_engine()

In [16]:
response = query_engine.query(
    "How should an organization manage user authorizations to ensure data security under GDPR?"
)
print(str(response))

In order to manage user authorizations for ensuring data security under GDPR, organizations need to adhere to certain principles and guidelines. These include the following steps:

1. Determine the scope of personal data processing: Organizations should clearly define the purposes and means of processing personal data. This involves identifying the categories of data subjects, types of personal data, and the processing activities involved in providing goods or services.

2. Appoint a representative for non-EU organizations: If an organization is not established within the European Union (EU) but processes personal data of EU residents, they are required to designate a representative unless their processing activities meet specific conditions. This representative should act on behalf of the controller or processor and be explicitly designated by a written mandate.

3. Ensure compliance through processors: When entrusting processing activities to a third-party service provider (processor

## Evaluation setup using TruLens

In [10]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

What are the essential security measures for effective teleworking under GDPR?
What organizational measures should be taken to ensure compliance with the GDPR in terms of data security?
What technical measures are essential for ensuring the security of personal data under GDPR?
What are the essential technical measures to secure equipment and workstations for protecting personal data?
How should an organization protect its premises to ensure the security of personal data?
How should an organization implement user authentication to protect personal data under GDPR requirements?
How should an organization manage user authorizations to ensure data security under GDPR?
What is pseudonymisation and how should it be implemented under GDPR?
How does encryption and hash functions contribute to GDPR compliance?
What does data anonymisation involve under GDPR, and how is it distinguished from pseudonymisation?


In [11]:
# You can try your own question:
new_question = "What are the essential security measures for effective teleworking under GDPR?"
eval_questions.append(new_question)

In [12]:
print(eval_questions)

['What are the essential security measures for effective teleworking under GDPR?', 'What organizational measures should be taken to ensure compliance with the GDPR in terms of data security?', 'What technical measures are essential for ensuring the security of personal data under GDPR?', 'What are the essential technical measures to secure equipment and workstations for protecting personal data?', 'How should an organization protect its premises to ensure the security of personal data?', 'How should an organization implement user authentication to protect personal data under GDPR requirements?', 'How should an organization manage user authorizations to ensure data security under GDPR?', 'What is pseudonymisation and how should it be implemented under GDPR?', 'How does encryption and hash functions contribute to GDPR compliance?', 'What does data anonymisation involve under GDPR, and how is it distinguished from pseudonymisation?', 'What are the essential security measures for effective

In [13]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


For the classroom, we've written some of the code in helper functions inside a utils.py file.  
- You can view the utils.py file in the file directory by clicking on the "Jupyter" logo at the top of the notebook.
- In later lessons, you'll get to work directly with the code that's currently wrapped inside these helper functions, to give you more options to customize your RAG pipeline.

In [14]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

In [15]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

In [16]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [17]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_62c341eb4737383f20816d6f4734d1de,"""What are the essential security measures for ...","""Essential security measures for effective tel...",-,"{""record_id"": ""record_hash_62c341eb4737383f208...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-06-05T14:54:43.741329"", ""...",2024-06-05T14:54:46.239427,1.0,0.00,0.88,[{'args': {'prompt': 'What are the essential s...,[{'args': {'prompt': 'What are the essential s...,[{'args': {'source': 'Where such notification...,2,2108,0.003197
1,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_9773fcab739cfe333c0c1faed8cd59b5,"""What organizational measures should be taken ...","""Organizational measures such as evaluating ri...",-,"{""record_id"": ""record_hash_9773fcab739cfe333c0...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-06-05T14:54:46.456717"", ""...",2024-06-05T14:54:48.790732,1.0,1.00,0.85,[{'args': {'prompt': 'What organizational meas...,[{'args': {'prompt': 'What organizational meas...,[{'args': {'source': 'Those measures shall be ...,2,2182,0.003299
2,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_fbec59609d2aca15b4d0db6ac64bf53f,"""What technical measures are essential for ens...","""Implementing appropriate technical and organi...",-,"{""record_id"": ""record_hash_fbec59609d2aca15b4d...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-06-05T14:54:49.006988"", ""...",2024-06-05T14:54:51.891685,1.0,0.55,1.00,[{'args': {'prompt': 'What technical measures ...,[{'args': {'prompt': 'What technical measures ...,[{'args': {'source': 'Those restr ictions shou...,2,2214,0.003362
3,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_5f0623e9ba4a591c6b018a575daf8ed2,"""What are the essential technical measures to ...","""The essential technical measures to secure eq...",-,"{""record_id"": ""record_hash_5f0623e9ba4a591c6b0...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-06-05T14:54:52.108464"", ""...",2024-06-05T14:54:54.642781,1.0,0.60,1.00,[{'args': {'prompt': 'What are the essential t...,[{'args': {'prompt': 'What are the essential t...,"[{'args': {'source': 'When developing, designi...",2,2204,0.003338
4,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_160d725106456c916e2ddbf588091ed8,"""How should an organization protect its premis...","""An organization should evaluate the risks inh...",-,"{""record_id"": ""record_hash_160d725106456c916e2...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-06-05T14:54:54.842720"", ""...",2024-06-05T14:54:57.066385,1.0,0.60,1.00,[{'args': {'prompt': 'How should an organizati...,[{'args': {'prompt': 'How should an organizati...,[{'args': {'source': 'The controller and proc...,2,2204,0.003340


In [19]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.178.104:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## Advanced RAG pipeline

### 1. Sentence Window retrieval

In [20]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [21]:
from utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [22]:
from utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

c:\Users\Simon\OneDrive\UNI\Semester 8\RAG\venv2\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Simon\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [24]:
window_response = sentence_window_engine.query(
    "How should an organization manage user authorizations to ensure data security under GDPR?"
)
print(str(window_response))

An organization should adopt internal policies and implement measures that meet the principles of data protection by design and data protection by default. This includes minimizing the processing of personal data, pseudonymizing personal data, ensuring transparency in data processing, enabling data subjects to monitor data processing, and creating and improving security features. Additionally, when developing applications, services, and products that involve processing personal data, organizations should consider data protection rights and ensure that controllers and processors can fulfill their data protection obligations. The principles of data protection by design and by default should also be considered in the context of public tenders.


In [25]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = "Sentence Window Query Engine"
)

In [26]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

What are the essential security measures for effective teleworking under GDPR?
The essential security measures for effective teleworking under GDPR include implementing appropriate technological protection and organizational measures to promptly detect any personal data breaches, informing the supervisory authority and the data subject without undue delay in case of a breach likely to result in a high risk to the rights and freedoms of the individual, and communicating the nature of the breach along with recommendations to mitigate potential adverse effects to the data subject as soon as reasonably feasible.
What organizational measures should be taken to ensure compliance with the GDPR in terms of data security?
Organizational measures that should be taken to ensure compliance with the GDPR in terms of data security include adopting internal policies, implementing measures that align with data protection principles, such as data protection by design and by default, minimizing the proc

In [27]:
tru.get_leaderboard(app_ids=[])

,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_id,,,,,
Sentence Window Query Engine,1.0,0.468182,0.701732,8.090909,0.001901


In [28]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.178.104:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### 2. Auto-merging retrieval

In [29]:
from utils import build_automerging_index

automerging_index = build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index"
)

In [30]:
from utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [31]:
auto_merging_response = automerging_query_engine.query(
    "How should an organization manage user authorizations to ensure data security under GDPR?"
)
print(str(auto_merging_response))

An organization should adopt internal policies and implement measures that meet the principles of data protection by design and data protection by default in order to manage user authorizations and ensure data security under GDPR.


In [32]:
tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(automerging_query_engine,
                                                         app_id="Automerging Query Engine")

In [33]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

What are the essential security measures for effective teleworking under GDPR?
The essential security measures for effective teleworking under GDPR include implementing technical and organizational measures to correct inaccuracies in personal data, minimize the risk of errors, and secure personal data in a way that considers potential risks to the interests and rights of data subjects. Additionally, measures should be in place to prevent discriminatory effects on individuals based on factors like racial or ethnic origin, political opinion, religion, or beliefs.
What organizational measures should be taken to ensure compliance with the GDPR in terms of data security?
Organizational measures such as adopting internal policies and implementing measures that adhere to the principles of data protection by design and data protection by default should be taken to ensure compliance with the GDPR in terms of data security. Additionally, safeguards should be in place to ensure technical and orga

In [34]:
tru.get_leaderboard(app_ids=[])

,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_id,,,,,
Automerging Query Engine,0.96,0.485,0.890741,5.272727,0.000631


In [35]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.178.104:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>